In [15]:
!pip install transformers keybert

In [29]:
#java 오류시
#import os
#os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-23'  # 설치된 JDK 경로로 변경
#os.environ['PATH'] = os.environ['JAVA_HOME'] + r'\bin;' + os.environ['PATH']

In [1]:
url = 'https://n.news.naver.com/mnews/article/014/0005261392'

In [37]:
import json
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from keybert import KeyBERT
import torch
import re

# FinBERT 모델과 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

# KeyBERT 초기화
kw_model = KeyBERT()

# 불용어 및 조사, 어미 리스트 설정
stop_words = {'입니다', '합니다', '있습니다', '그것', '그러한', '또한', '이러한', '있도록', '위해',
              '하는', '에', '은', '는', '가', '을', '를', '으로', '에서', '및', '와', '과', '의', '도', 
              '한다고', '합니다', '하고', '하며'}

# 조사 및 어미를 제거하기 위한 간단한 패턴
suffix_pattern = re.compile(r'(에|에서|으로|라고|이라고|도|과|와|들|에다가|에도|도|하게|하게도|한테|에게|으로서|로서|하는|하며|되고|이라고|하면서|했다고)$')

# 주어진 URL에서 기사 본문을 크롤링하는 함수
def crawl_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # 'article' 태그의 'id="dic_area"'를 선택하여 본문을 가져옴
    article_content = soup.select_one('article#dic_area')
    return article_content.get_text(separator=" ", strip=True) if article_content else None

# FinBERT를 사용하여 금융 관련 키워드 추출
def extract_financial_terms(text, top_n=10):
    # KeyBERT를 사용해 주요 키워드 추출
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=top_n)
    
    # FinBERT로 키워드를 검토하여 금융 용어 필터링
    financial_terms = set()  # 중복 제거를 위해 set 사용
    for kw in keywords:
        inputs = tokenizer(kw[0], return_tensors="pt")
        outputs = model(**inputs)
        scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
        label = torch.argmax(scores).item()
        
        # 긍정(Positive), 부정(Negative), 중립(Neutral) 중 금융 관련성이 높은 용어를 추가
        if label == 0:  # 중립 (금융 용어 가능성이 높음)
            # 키워드가 여러 단어로 구성된 경우 분리하여 추가
            words = kw[0].split()
            for word in words:
                # 불용어에 없는 단어만 추가
                if word not in stop_words:
                    # 조사 및 어미 제거
                    cleaned_word = suffix_pattern.sub('', word)
                    financial_terms.add(cleaned_word)

    return list(financial_terms)

# 노드와 관계를 구조화하여 JSON 형식으로 저장
def create_nodes_and_edges(terms):
    node_id = 1
    nodes = []
    edges = []

    term_map = {}
    for term in terms:
        # 노드 생성
        nodes.append({"id": node_id, "word": term})
        term_map[term] = node_id
        node_id += 1

    # 용어 간 임의 관계 추가 (예시)
    for i in range(len(terms)):
        for j in range(i + 1, len(terms)):
            edges.append({
                "source": term_map[terms[i]],
                "target": term_map[terms[j]],
                "relation_description": f"{terms[i]}와 {terms[j]}는 금융 기사에서 관련된 용어로 함께 등장합니다."
            })

    return {"nodes": nodes, "edges": edges}

# 기사 본문 크롤링
url = "https://n.news.naver.com/mnews/article/014/0005261392"  # 분석할 URL을 입력하세요
article_text = crawl_article_text(url)

# 본문이 없을 경우 종료
if not article_text:
    print(f"Article content not found for {url}")
else:
    # FinBERT 및 KeyBERT를 통해 금융 용어 추출
    financial_terms = extract_financial_terms(article_text)

    # 노드 및 관계 생성
    extracted_data = create_nodes_and_edges(financial_terms)

    # 결과를 출력
    print("Extracted Financial Terms and Relationships:")
    print(json.dumps(extracted_data, ensure_ascii=False, indent=4))


Extracted Financial Terms and Relationships:
{
    "nodes": [
        {
            "id": 1,
            "word": "사용자들이"
        },
        {
            "id": 2,
            "word": "투입한다고"
        },
        {
            "id": 3,
            "word": "운영하는"
        },
        {
            "id": 4,
            "word": "ai"
        },
        {
            "id": 5,
            "word": "소셜미디어에"
        },
        {
            "id": 6,
            "word": "행사에서"
        },
        {
            "id": 7,
            "word": "사업에서"
        },
        {
            "id": 8,
            "word": "시대"
        },
        {
            "id": 9,
            "word": "부문에서"
        },
        {
            "id": 10,
            "word": "페이스북과"
        },
        {
            "id": 11,
            "word": "ai의"
        },
        {
            "id": 12,
            "word": "인공지능"
        },
        {
            "id": 13,
            "word": "기술의"
        },
        {
            "id": 14,
      

In [ ]:
gnn_enhanced_700_dict.json

In [ ]:
gnn을 학습을 시키고 -> 노드간의 적절한 관게를 gnn이 생성할 수 있다면 키워드만 뽑으면 된다 | 절대 안됨 이거
gnn이 학습할 수 있도록 추론(?) 